In [8]:
# 画骑手轨迹，用于Return Path的motivation
# Yi Ding
# 07/17/18

import matplotlib.pyplot as plt
import gmplot
import pandas as pd
import os

In [11]:
# Read in data
df_rider_trace= pd.read_excel(
    '/Users/eleme-yi/Documents/PhD/GitHub Pages/dymodi.github.io/Research/Paper/Rider-Routing/data/rider_trace_10820633.xlsx')
print(df_rider_trace.head())
figure_path = '/Users/eleme-yi/Documents/PhD/GitHub Pages/dymodi.github.io/Research/Paper/Rider-Routing/figures/'
figure_name = os.path.join(figure_path, 'rider_trace_10820633.html')

   taker_id            record_time  time_stamp   longitude   latitude  \
0  10820633  2018-07-16 08:34:15.0  1531701255  121.426871  31.293167   
1  10820633  2018-07-16 08:35:37.0  1531701337  121.426871  31.293167   
2  10820633  2018-07-16 08:38:45.0  1531701525  121.426899  31.293202   
3  10820633  2018-07-16 08:41:45.0  1531701705  121.426914  31.293196   
4  10820633  2018-07-16 08:44:45.0  1531701885  121.426920  31.293189   

                                              detail  zone_id  city_id  \
0  {"speed": 0, "course": 0, "altitude": 0, "lati...        0        0   
1  {"speed": 0, "course": 0, "altitude": 0, "lati...        0        0   
2  {"speed": 0, "course": 0, "altitude": 29.7, "l...        0        0   
3  {"speed": 0, "course": 0, "altitude": 32.6, "l...        0        0   
4  {"speed": 0, "course": 0, "altitude": 34.6, "l...        0        0   

           dt          type  
0  2018-07-16  TALARIS_TEAM  
1  2018-07-16  TALARIS_TEAM  
2  2018-07-16  TALARIS_TEA

In [13]:
# Plot trace
gmap = gmplot.GoogleMapPlotter(df_rider_trace['latitude'].iloc[0], df_rider_trace['longitude'].iloc[0], 16)
# gmap.plot(df_rider_trace['latitude'].tolist(), df_rider_trace['longitude'].tolist(), 'blue', edge_width=10)
gmap.scatter(df_rider_trace['latitude'].tolist(), df_rider_trace['longitude'].tolist(), 'green', marker=True)
gmap.draw(figure_name)